In [1]:
# 코드 10-25 라이브러리 호출
from __future__ import unicode_literals, print_function, division  # ①
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re  # ②
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 코드 10-26 데이터 준비
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20


class Lang:  # 딕셔너리를 만들기 위한 클래스
    def __init__(self):  # 단어의 인덱스를 저장하기 위한 컨테이너를 초기화
        self.word2index = {}
        self.word2count = {}
        self.index2word = {
            0: "SOS",
            1: "EOS",
        }  # SOS(Start Of Sequence): 문장의 시작, EOS(End Of Sequence): 문장의 끝
        self.n_words = 2  # SOS와 EOS에 대한 카운트

    def addSentence(
        self, sentence
    ):  # 문장을 단어 단위로 분리한 후 컨테이너(word)에 추가
        for word in sentence.split(" "):
            self.addWord(word)

    def addWord(
        self, word
    ):  # 컨테이너에 단어가 없다면 추가되고, 있다면 카운트를 업데이트
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# 코드 10-27 데이터 정규화
def normalizeString(df, lang):
    sentence = df[lang].str.lower()  # 소문자로 전환
    sentence = sentence.str.replace(
        "[^A-Za-z\s]+", " "
    )  # a-z, A-Z, …, ?, ! 등을 제외하고 모두 공백으로 바꿈
    sentence = sentence.str.normalize("NFD")  # 유니코드 정규화 방식
    sentence = sentence.str.encode("ascii", errors="ignore").str.decode(
        "utf-8"
    )  # Unicode를 ASCII로 전환
    return sentence


def read_sentence(df, lang1, lang2):
    sentence1 = normalizeString(df, lang1)  # 데이터셋의 첫 번째 열(영어)
    sentence2 = normalizeString(df, lang2)  # 데이터셋의 두 번째 열(프랑스어)
    return sentence1, sentence2


def read_file(loc, lang1, lang2):
    df = pd.read_csv(loc, delimiter="\t", header=None, names=[lang1, lang2])  # ①
    return df


def process_data(lang1, lang2):  # 데이터셋 불러오기
    df = read_file("../chap10/data/%s-%s.txt" % (lang1, lang2), lang1, lang2)
    sentence1, sentence2 = read_sentence(df, lang1, lang2)

    input_lang = Lang()
    output_lang = Lang()
    pairs = []
    for i in range(len(df)):
        if (
            len(sentence1[i].split(" ")) < MAX_LENGTH
            and len(sentence2[i].split(" ")) < MAX_LENGTH
        ):
            full = [sentence1[i], sentence2[i]]  # 첫 번째와 두 번째 열을 합쳐서 저장
            input_lang.addSentence(sentence1[i])  # 입력(input)으로 영어를 사용
            output_lang.addSentence(sentence2[i])  # 출력(output)으로 프랑스어를 사용
            pairs.append(full)  # pairs에는 입력과 출력이 합쳐진 것을 사용

    return input_lang, output_lang, pairs

In [4]:
# 코드 10-28 텐서로 변환
def indexesFromSentence(lang, sentence):  # 문장을 단어로 분리하고 인덱스를 반환
    return [lang.word2index[word] for word in sentence.split(" ")]


def tensorFromSentence(
    lang, sentence
):  # 딕셔너리에서 단어에 대한 인덱스를 가져오고 문장 끝에 토큰을 추가
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(
    input_lang, output_lang, pair
):  # 입력과 출력 문장을 텐서로 변환하여 반환
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [5]:
# 코드 10-29 인코더 네트워크
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()
        self.input_dim = input_dim  # 인코더에서 사용할 입력층
        self.embbed_dim = embbed_dim  # 인코더에서 사용할 임베딩 계층
        self.hidden_dim = hidden_dim  # 인코더에서 사용할 은닉층(이전 은닉층)
        self.num_layers = num_layers  # 인코더에서 사용할 GRU의 계층 개수
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)  # 임베딩 계층 초기화
        self.gru = nn.GRU(
            self.embbed_dim, self.hidden_dim, num_layers=self.num_layers
        )  # 임베딩 차원, 은닉층 차원, GRU의 계층 개수를 이용하여 GRU 계층을 초기화

    def forward(self, src):
        embedded = self.embedding(src).view(1, 1, -1)  # 임베딩 처리
        outputs, hidden = self.gru(embedded)  # 임베딩 결과를 GRU 모델에 적용
        return outputs, hidden

In [6]:
# 코드 10-30 디코더 네트워크
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)  # 임베딩 계층 초기화
        self.gru = nn.GRU(
            self.embbed_dim, self.hidden_dim, num_layers=self.num_layers
        )  # GRU 계층 초기화
        self.out = nn.Linear(self.hidden_dim, output_dim)  # 선형 계층 초기화
        self.softmax = nn.LogSoftmax(dim=1)  # ①

    def forward(self, input, hidden):
        input = input.view(1, -1)  # 입력을 (1, 배치 크기)로 변경
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden

In [7]:
# 코드 10-31 seq2seq 네트워크
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()

        self.encoder = encoder  # 인코더 초기화
        self.decoder = decoder  # 디코더 초기화
        self.device = device

    def forward(self, input_lang, output_lang, teacher_forcing_ratio=0.5):
        input_length = input_lang.size(0)  # 입력 문자 길이(문장의 단어 수)
        batch_size = output_lang.shape[1]
        target_length = output_lang.shape[0]
        vocab_size = self.decoder.output_dim
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(
            self.device
        )  # 예측된 출력을 저장하기 위한 변수 초기화

        for i in range(input_length):
            encoder_output, encoder_hidden = self.encoder(
                input_lang[i]
            )  # 문장의 모든 단어를 인코딩
        decoder_hidden = encoder_hidden.to(
            device
        )  # 인코더의 은닉층을 디코더의 은닉층으로 사용
        decoder_input = torch.tensor(
            [SOS_token], device=device
        )  # 첫 번째 예측 단어 앞에 토큰(SOS) 추가

        for t in range(target_length):  # 현재 단어에서 출력 단어를 예측
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio  # ①
            topv, topi = decoder_output.topk(1)
            input = (
                output_lang[t] if teacher_force else topi
            )  # teacher_force를 활성화하면 목표 단어를 다음 입력으로 사용
            if (
                teacher_force == False and input.item() == EOS_token
            ):  # teacher_force를 활성화하지 않으면 자체 예측 값을 다음 입력으로 사용
                break
        return outputs

In [8]:
# 코드 10-32 모델의 오차 계산 함수 정의
teacher_forcing_ratio = 0.5


def Model(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0
    output = model(input_tensor, target_tensor)
    num_iter = output.size(0)

    for ot in range(num_iter):
        loss += criterion(
            output[ot], target_tensor[ot]
        )  # 모델의 예측 결과와 정답(예상 결과)을 이용하여 오차를 계산
    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter
    return epoch_loss

In [9]:
# 코드 10-33 모델 훈련 함수 정의
def trainModel(model, input_lang, output_lang, pairs, num_iteration=20000):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=0.01)  # 옵티마이저로 SGD를 사용
    criterion = nn.NLLLoss()  # ①
    total_loss_iterations = 0

    training_pairs = [
        tensorsFromPair(input_lang, output_lang, random.choice(pairs))
        for i in range(num_iteration)
    ]

    for iter in range(1, num_iteration + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = Model(
            model, input_tensor, target_tensor, optimizer, criterion
        )  # Model 객체를 이용하여 오차 계산
        total_loss_iterations += loss

        if iter % 5000 == 0:  # 5000번째마다 오차 값에 대해 출력
            average_loss = total_loss_iterations / 5000
            total_loss_iterations = 0
            print("%d %.4f" % (iter, average_loss))

    torch.save(model.state_dict(), "../chap10/data/mytraining.pt")
    return model

In [10]:
# 코드 10-34 모델 평가
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(
            input_lang, sentences[0]
        )  # 입력 문자열을 텐서로 변환
        output_tensor = tensorFromSentence(
            output_lang, sentences[1]
        )  # 출력 문자열을 텐서로 변환
        decoded_words = []
        output = model(input_tensor, output_tensor)

        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(
                1
            )  # 각 출력에서 가장 높은 값을 찾아 인덱스를 반환

            if topi[0].item() == EOS_token:
                decoded_words.append("<EOS>")  # EOS 토큰을 만나면 평가를 멈춥니다.
                break
            else:
                decoded_words.append(
                    output_lang.index2word[topi[0].item()]
                )  # 예측 결과를 출력 문자열에 추가

    return decoded_words


def evaluateRandomly(
    model, input_lang, output_lang, pairs, n=10
):  # 훈련 데이터셋으로부터 임의의 문장을 가져와서 모델 평가
    for i in range(n):
        pair = random.choice(pairs)  # 임의로 문장을 가져옵니다.
        print("input {}".format(pair[0]))
        print("output {}".format(pair[1]))
        output_words = evaluate(
            model, input_lang, output_lang, pair
        )  # 모델 평가 결과는 output_words에 저장
        output_sentence = " ".join(output_words)
        print("predicted {}".format(output_sentence))

In [11]:
# 코드 10-35 모델 훈련
lang1 = "eng"  # 입력으로 사용할 영어
lang2 = "fra"  # 출력으로 사용할 프랑스어
input_lang, output_lang, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print("random sentence {}".format(randomize))

input_size = input_lang.n_words
output_size = output_lang.n_words
print(
    "Input : {} Output : {}".format(input_size, output_size)
)  # 입력과 출력에 대한 단어 수 출력

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 75000  # 7만 5000번 반복하여 모델 훈련

encoder = Encoder(
    input_size, hidden_size, embed_size, num_layers
)  # 인코더에 훈련 데이터셋을 입력하고 모든 출력과 은닉 상태를 저장
decoder = Decoder(
    output_size, hidden_size, embed_size, num_layers
)  # 디코더의 첫 번째 입력으로 <SOS> 토큰이 제공되고, 인코더의 마지막 은닉 상태가 디코더의 첫 번째 은닉 상태로 제공됩니다.
model = Seq2Seq(encoder, decoder, device).to(
    device
)  # 인코더-디코더 모델(seq2seq)의 객체 생성

print(encoder)
print(decoder)

model = trainModel(model, input_lang, output_lang, pairs, num_iteration)  # 모델 학습

random sentence ['let me tell you everything i know.', 'laissez-moi vous conter tout ce que je sais.']
Input : 23191 Output : 39387
Encoder(
  (embedding): Embedding(23191, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(39387, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=39387, bias=True)
  (softmax): LogSoftmax(dim=1)
)
5000 5.0150
10000 4.8061
15000 4.7233
20000 4.6867
25000 4.6432
30000 4.6151
35000 4.6661
40000 4.6916
45000 4.6349
50000 4.6210
55000 4.5618
60000 4.5624
65000 4.5336
70000 4.5871
75000 4.6190


In [12]:
# 코드 10-36 임의의 문장에 대한 평가 결과
evaluateRandomly(model, input_lang, output_lang, pairs)

input don't let your mother know that.
output que ta mere n'en sache rien !
predicted je ne pas pas <EOS>
input why are you picking a fight with me?
output pourquoi me cherchez-vous querelle ?
predicted je ne pas pas <EOS>
input he was unconscious of his guilt.
output il etait inconscient de sa culpabilite.
predicted je ne pas pas <EOS>
input do you still want me to give tom your old computer?
output veux-tu toujours que je donne ton ancien ordinateur a tom ?
predicted je ne pas pas <EOS>
input i wish i had a friend like you.
output j'aimerais avoir une amie telle que vous.
predicted je ne pas pas <EOS>
input i love to party.
output j'adore faire la java.
predicted je ne pas pas <EOS>
input could you take our picture?
output pourrais-tu nous prendre en photo ?
predicted je ne pas pas <EOS>
input the telephone is ringing, but nobody is answering.
output le telephone sonne mais personne ne repond.
predicted je ne pas pas <EOS>
input are you sure that you haven't forgotten anything?
outpu

In [13]:
# 코드 10-37 어텐션이 적용된 디코더
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(
            self.output_size, self.hidden_size
        )  # 임베딩 계층 초기화
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)  # ①
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1
        )
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)
        )  # ②

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

In [14]:
# 코드 10-38 어텐션 디코더 모델 학습을 위한 함수
def trainIters(
    encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.SGD(
        encoder.parameters(), lr=learning_rate
    )  # 인코더와 디코더에 SGD 옵티마이저 적용
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [
        tensorsFromPair(input_lang, output_lang, random.choice(pairs))
        for i in range(n_iters)
    ]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]  # 입력+출력 쌍에서 입력을 input_tensor로 사용
        target_tensor = training_pair[1]  # 입력+출력 쌍에서 출력을 target_tensor로 사용
        loss = Model(model, input_tensor, target_tensor, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if (
            iter % 5000 == 0
        ):  # 모델을 7만 5000번 훈련을 진행하며 5000번째마다 오차를 출력
            print_loss_avg = print_loss_total / 5000
            print_loss_total = 0
            print("%d, %.4f" % (iter, print_loss_avg))

In [15]:
# 코드 10-39 어텐션 디코더 모델 훈련
import time

embed_size = 256
hidden_size = 512
num_layers = 1
input_size = input_lang.n_words
output_size = output_lang.n_words

encoder1 = Encoder(input_size, hidden_size, embed_size, num_layers)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_size, dropout_p=0.1).to(device)
print(encoder1)
print(attn_decoder1)

attn_model = trainIters(
    encoder1, attn_decoder1, 75000, print_every=5000, plot_every=100, learning_rate=0.01
)  # 인코더와 어텐션 디코더를 이용한 모델 생성

Encoder(
  (embedding): Embedding(23191, 256)
  (gru): GRU(256, 512)
)
AttnDecoderRNN(
  (embedding): Embedding(39387, 512)
  (attn): Linear(in_features=1024, out_features=20, bias=True)
  (attn_combine): Linear(in_features=1024, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(512, 512)
  (out): Linear(in_features=512, out_features=39387, bias=True)
)
5000, 4.9127
10000, 4.9277
15000, 4.9455
20000, 4.9397
25000, 4.9240
30000, 4.9247
35000, 4.9373
40000, 4.9325
45000, 4.9231
50000, 4.9487
55000, 4.9132
60000, 4.9453
65000, 4.8964
70000, 4.9250
75000, 4.9165
